## WELCOME TO THE FINALE - PROJECT 3

And

# WELCOME TO THE **M**ODEL **C**ONTEXT **P**ROTOCOL!

And welcome back to OpenAI Agents SDK

# Please consider this next hour to be a 'teaser'

We will fly by a lot of functionality; gain intuition, come back later



<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">To Windows PC people - known issue with MCP servers on PCs</h2>
            <span style="color:#ff7800;">I have unpleasant news. There's a problem running MCP Servers on Windows PCs; Mac and Linux is fine. This is a known issue as of May 4th, 2025. I asked o3 with Deep Research to try to find workarounds; it <a href="https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">confirmed the issue</a> and confirmed the workaround.<br/><br/>
            The workaround is a bit of a bore. It is to take advantage of "WSL", the Microsoft approach for running Linux on your PC. You'll need to carry out more setup instructions! But it's quick, and several students have confirmed that this works perfectly for them, then the Week 6 MCP labs work. Plus, WSL is actually a great way to build software on your Windows PC.<br/>
            The WSL Setup instructions are in the Setup folder, <a href="../setup/SETUP-WSL.md">in the file called SETUP-WSL.md here</a>. I do hope this only holds you up briefly - you should be back up and running quickly. Oh the joys of working with bleeding-edge technology!<br/><br/>
            With many thanks to students Markus, Abhi, Hui-Ling, and several others, for helping me work on it and confirming the fix.
            </span>
        </td>
    </tr>
</table>

In [1]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os
from IPython.display import Markdown, display
load_dotenv(override=True)


True

You may need to install node if you don't have it already.

https://chatgpt.com/share/68103af2-e2dc-8012-b259-bc135a23273b

### Let's use MCP in OpenAI Agents SDK

Fetch MCP tools: runs a headless browser controlled by Playwright

Filesystem MCP tools: can read and write from a chosen directory on your computer

Let's put them together to carry out an important task.

In [5]:
# A python based MCP server

fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get the actual H

In [6]:
# A JavaScript based MCP server

sandbox_path = os.path.abspath(os.path.join(os.getcwd(), "sandbox"))
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params) as server:
    file_tools = await server.list_tools()

file_tools

[Tool(name='read_file', description='Read the complete contents of a file from the file system. Handles various text encodings and provides detailed error messages if the file cannot be read. Use this tool when you need to examine the contents of a single file. Only works within allowed directories.', inputSchema={'type': 'object', 'properties': {'path': {'type': 'string'}}, 'required': ['path'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, annotations=None),
 Tool(name='read_multiple_files', description="Read the contents of multiple files simultaneously. This is more efficient than reading files one by one when you need to analyze or compare multiple files. Each file's content is returned with its path as a reference. Failed reads for individual files won't stop the entire operation. Only works within allowed directories.", inputSchema={'type': 'object', 'properties': {'paths': {'type': 'array', 'items': {'type': 'string'}}}, 'required': ['pat

In [4]:
instructions = """
You browse the internet to accomplish your instructions.
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""


async with MCPServerStdio(params=fetch_params) as fetch_mcp_server:
    async with MCPServerStdio(params=files_params) as files_mcp_server:
        agent = Agent(
            name="investigator", 
            instructions=instructions, 
            model="gpt-4o-mini",
            mcp_servers=[fetch_mcp_server, files_mcp_server]
            )
        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md")
            display(Markdown(result.final_output))

I've created a markdown file for the Banoffee Pie recipe. You can find it at `sandbox/banoffee.md`. Here's a summary of the recipe:

```markdown
# Banoffee Pie Recipe

## Ingredients
- **For the pastry:**  
  - Butter  
  - Flour  
  - Egg yolk  
  - Sugar  
  - Very cold water  

- **For the filling:**  
  - Caramel (half for the base, half for layering)  
  - Bananas (sliced)  
  - Whipped cream  
  - Dark chocolate (for garnish)  

## Method
1. **Make the pastry case:**  
   - In a food processor, pulse butter and flour until it resembles fresh breadcrumbs.  
   - Add egg yolk and sugar, then pulse again. Gradually add very cold water until it forms a ball of dough.  
   - Knead gently then wrap in cling film and chill in the fridge for 30 mins.

2. **Prepare for baking:**  
   - Preheat the oven to 190C/170C/gas 4. Grease a 23cm loose-bottomed tart tin.  
   - Roll out the pastry and line the tin, snipping excess pastry edge.  
   - Line with baking parchment and weigh down with baking beans. Bake for 15 mins, then remove the paper and beans, brush with egg white, and bake for an additional 15-20 mins until golden. Cool completely.

3. **Assemble the pie:**  
   - Spread half of the caramel on the cooled pastry case and layer sliced bananas on top.  
   - Cover with the remaining caramel and chill in the fridge.

4. **Finish off:**  
   - Whip the cream until thick and spoon over the tart.  
   - Garnish with finely grated dark chocolate before serving.
```

Feel free to check the file or modify it as needed!